In [22]:
import pandas as pd
from collections import defaultdict

# =========================
# AYARLAR
# =========================
MAX_SCORE_BY_TYPE = {
    "Large Garbage Truck": 2.5,
    "Small Garbage Truck": 1.8,
    "Crane Vehicle": 2.8
}

VEHICLE_START_PENALTY = 0.3
MAX_NEIGHBORHOODS_PER_VEHICLE = 12

# =========================
# 1. VERİLERİ OKU
# =========================
service_plan = pd.read_csv("service_plan_by_date.csv")
demand = pd.read_csv("demand_by_date_neighborhood.csv")
fleet = pd.read_csv("fleet.csv")
truck_types = pd.read_csv("truck_types.csv")

# =========================
# 2. NORMALİZASYON
# =========================
def norm(x):
    return str(x).strip().lower()

service_plan["Mahalle_norm"] = service_plan["Mahalle"].apply(norm)
demand["Mahalle_norm"] = demand["Mahalle"].apply(norm)

# =========================
# 3. JOB TABLOSU (FAZ 4 İŞ TANIMI)
# =========================
jobs = service_plan.merge(
    demand[["date", "Mahalle_norm", "demand_score"]],
    on=["date", "Mahalle_norm"],
    how="left"
)

jobs["demand_score"] = jobs["demand_score"].fillna(0)

def resolve_required_type(row):
    if row["crane_needed"] == 1:
        return "Crane Vehicle"
    return row["garbage_truck_type"]

jobs["required_vehicle_type"] = jobs.apply(resolve_required_type, axis=1)

# =========================
# 4. FLEET HAZIRLIĞI
# =========================
fleet = fleet.copy()
fleet["vehicle_type"] = fleet["vehicle_type"].astype(str)

if "active" in fleet.columns:
    fleet = fleet[fleet["active"] == 1]

# =========================
# 5. GÜN BAZLI ASSIGNMENT
# =========================
assignments = []
unassigned = []
day_summary = []

for date, day_jobs in jobs.groupby("date"):

    day_jobs = day_jobs.sort_values("demand_score", ascending=False)

    # araç durumları
    vehicles = []
    for _, v in fleet.iterrows():
        vehicles.append({
            "vehicle_id": v["vehicle_id"],
            "vehicle_type": v["vehicle_type"],
            "load": 0.0,
            "jobs": []
        })

    used_vehicle_ids = set()

    for _, job in day_jobs.iterrows():
        req_type = job["required_vehicle_type"]
        score = job["demand_score"]

        candidates = [
            v for v in vehicles
            if v["vehicle_type"] == req_type
            and v["load"] + score <= MAX_SCORE_BY_TYPE.get(req_type, 999)
            and len(v["jobs"]) < MAX_NEIGHBORHOODS_PER_VEHICLE
        ]

        if not candidates:
            unassigned.append({
                "date": date,
                "Mahalle": job["Mahalle"],
                "required_vehicle_type": req_type,
                "demand_score": score,
                "reason": f"no_available_{req_type}"
            })
            continue

        def cost(v):
            load_ratio = v["load"] / MAX_SCORE_BY_TYPE.get(req_type, 1)
            start_penalty = VEHICLE_START_PENALTY if v["vehicle_id"] not in used_vehicle_ids else 0
            return load_ratio + start_penalty

        chosen = min(candidates, key=cost)

        chosen["load"] += score
        chosen["jobs"].append(job["Mahalle"])
        used_vehicle_ids.add(chosen["vehicle_id"])

        assignments.append({
            "date": date,
            "vehicle_id": chosen["vehicle_id"],
            "vehicle_type": chosen["vehicle_type"],
            "Mahalle": job["Mahalle"],
            "demand_score": score,
            "crane_needed": job["crane_needed"]
        })

    # GÜNLÜK ÖZET
    total_jobs = len(day_jobs)
    assigned_jobs = sum(len(v["jobs"]) for v in vehicles)
    total_demand = day_jobs["demand_score"].sum()
    assigned_demand = sum(v["load"] for v in vehicles)

    day_summary.append({
        "date": date,
        "jobs_total": total_jobs,
        "jobs_assigned": assigned_jobs,
        "jobs_unassigned": total_jobs - assigned_jobs,
        "vehicles_used": len(used_vehicle_ids),
        "demand_total": total_demand,
        "demand_assigned": assigned_demand,
        "demand_unassigned": total_demand - assigned_demand
    })

# =========================
# 6. CSV ÇIKTILARI
# =========================
pd.DataFrame(assignments).to_csv("phase4_assignments.csv", index=False)
pd.DataFrame(unassigned).to_csv("phase4_unassigned.csv", index=False)
pd.DataFrame(day_summary).to_csv("phase4_day_summary.csv", index=False)

print("FAZ 4.0 TAMAMLANDI")
print("-> phase4_assignments.csv")
print("-> phase4_unassigned.csv")
print("-> phase4_day_summary.csv")


FAZ 4.0 TAMAMLANDI
-> phase4_assignments.csv
-> phase4_unassigned.csv
-> phase4_day_summary.csv


In [19]:
a = pd.read_csv("phase4_assignments.csv")
print("Crane needed ama crane araç değil (hata) sayısı:",
      ((a["crane_needed"]==1) & (a["vehicle_is_crane"]!=1)).sum())


Crane needed ama crane araç değil (hata) sayısı: 0


In [ ]:
# Hücre 1 — Dosyaları oku
import pandas as pd
from pathlib import Path

BASE_DIR = Path(".")

OUT_ASSIGN = BASE_DIR / "phase4_assignments.csv"
OUT_SUMM   = BASE_DIR / "phase4_day_summary.csv"

assign_df = pd.read_csv(OUT_ASSIGN)
summ_df   = pd.read_csv(OUT_SUMM)

print("OK: Dosyalar okundu.")
print("assign_df:", assign_df.shape, "| summ_df:", summ_df.shape)

In [4]:
# Hücre 2 — phase4_day_summary.csv: jobs_unassigned 0 mı? (gün gün kontrol)
if "jobs_unassigned" not in summ_df.columns:
    raise ValueError("summ_df içinde 'jobs_unassigned' kolonu yok. Kolonlar: " + ", ".join(summ_df.columns))

bad_days = summ_df[summ_df["jobs_unassigned"] != 0].copy()

print("jobs_unassigned tüm günlerde 0 mı? =>", bad_days.empty)

if not bad_days.empty:
    print("\njobs_unassigned != 0 olan günler (problem var):")
    display(bad_days[["date", "jobs_total", "jobs_assigned", "jobs_unassigned", "active_vehicles"]]
            .sort_values("jobs_unassigned", ascending=False))


jobs_unassigned tüm günlerde 0 mı? => False

jobs_unassigned != 0 olan günler (problem var):


,date,jobs_total,jobs_assigned,jobs_unassigned,active_vehicles
1,2025-12-20,32,24,8,24
6,2025-12-25,32,24,8,24
4,2025-12-23,25,18,7,18
0,2025-12-19,40,35,5,25
3,2025-12-22,40,35,5,25
5,2025-12-24,40,35,5,25


In [5]:
# Hücre 3 — jobs_unassigned 0 değilse hızlı teşhis: atanamayanların nedenleri + req_truck_type/crane_needed dağılımı
# (Bu hücreyi sadece Hücre 2'de problem çıktıysa çalıştırman mantıklı)

OUT_UNASSG = BASE_DIR / "phase4_unassigned.csv"
unassg_df = pd.read_csv(OUT_UNASSG)

print("unassg_df:", unassg_df.shape)

# neden dağılımı
if "reason" in unassg_df.columns:
    print("\nReason dağılımı:")
    print(unassg_df["reason"].value_counts(dropna=False))

# crane_needed ve req_truck_type kırılımı
for col in ["crane_needed", "req_truck_type"]:
    if col in unassg_df.columns:
        print(f"\n{col} dağılımı:")
        print(unassg_df[col].value_counts(dropna=False))


unassg_df: (38, 7)

Reason dağılımı:
reason
no_feasible_vehicle    38
Name: count, dtype: int64

crane_needed dağılımı:
crane_needed
1    38
Name: count, dtype: int64

req_truck_type dağılımı:
req_truck_type
large    38
Name: count, dtype: int64


In [6]:
# Hücre 4 — phase4_assignments.csv: crane_needed=1 olanların hepsi vehicle_is_crane=1 mi?
required_cols = {"crane_needed", "vehicle_is_crane"}
missing = required_cols - set(assign_df.columns)
if missing:
    raise ValueError(f"assign_df içinde eksik kolon(lar): {missing}. Kolonlar: {list(assign_df.columns)}")

crane_jobs = assign_df[assign_df["crane_needed"] == 1].copy()
violations = crane_jobs[crane_jobs["vehicle_is_crane"] != 1].copy()

print("crane_needed=1 job sayısı:", len(crane_jobs))
print("İhlal var mı? (crane_needed=1 iken vehicle_is_crane!=1) =>", not violations.empty)

if not violations.empty:
    print("\nİhlal örnekleri (ilk 20):")
    display(violations.head(20))


crane_needed=1 job sayısı: 0
İhlal var mı? (crane_needed=1 iken vehicle_is_crane!=1) => False


In [7]:
# Hücre 5 — active_vehicles çok mu yüksek? (günlük dağılım + temel istatistik)
if "active_vehicles" not in summ_df.columns:
    raise ValueError("summ_df içinde 'active_vehicles' kolonu yok. Kolonlar: " + ", ".join(summ_df.columns))

desc = summ_df["active_vehicles"].describe()
print("active_vehicles istatistik:")
print(desc)

print("\nGünlük active_vehicles (yüksekten düşüğe ilk 15):")
display(summ_df[["date", "jobs_total", "jobs_assigned", "active_vehicles", "demand_coverage"]]
        .sort_values("active_vehicles", ascending=False)
        .head(15))


active_vehicles istatistik:
count     7.000000
mean     20.428571
std       8.502101
min       2.000000
25%      21.000000
50%      24.000000
75%      25.000000
max      25.000000
Name: active_vehicles, dtype: float64

Günlük active_vehicles (yüksekten düşüğe ilk 15):


,date,jobs_total,jobs_assigned,active_vehicles,demand_coverage
0,2025-12-19,40,35,25,0.816466
3,2025-12-22,40,35,25,0.816466
5,2025-12-24,40,35,25,0.816466
1,2025-12-20,32,24,24,0.633000
6,2025-12-25,32,24,24,0.633000
4,2025-12-23,25,18,18,0.511570
2,2025-12-21,2,2,2,1.000000


In [8]:
# Hücre 6 — “çok yüksek mi?” için pratik oran: jobs_assigned / active_vehicles
# (Araç başına kaç iş düşüyor? Düşükse araç sayısı gereğinden fazla yayılıyor olabilir.)
if not set(["jobs_assigned", "active_vehicles"]).issubset(summ_df.columns):
    raise ValueError("summ_df içinde jobs_assigned veya active_vehicles eksik.")

tmp = summ_df.copy()
tmp["jobs_per_active_vehicle"] = tmp["jobs_assigned"] / tmp["active_vehicles"].replace(0, pd.NA)

print("Araç başına iş (jobs_per_active_vehicle) istatistik:")
print(tmp["jobs_per_active_vehicle"].describe())

print("\nAraç başına iş en düşük günler (en kötü 15):")
display(tmp[["date", "jobs_assigned", "active_vehicles", "jobs_per_active_vehicle"]]
        .sort_values("jobs_per_active_vehicle", ascending=True)
        .head(15))


Araç başına iş (jobs_per_active_vehicle) istatistik:
count    7.000000
mean     1.171429
std      0.213809
min      1.000000
25%      1.000000
50%      1.000000
75%      1.400000
max      1.400000
Name: jobs_per_active_vehicle, dtype: float64

Araç başına iş en düşük günler (en kötü 15):


,date,jobs_assigned,active_vehicles,jobs_per_active_vehicle
1,2025-12-20,24,24,1.0
2,2025-12-21,2,2,1.0
4,2025-12-23,18,18,1.0
6,2025-12-25,24,24,1.0
0,2025-12-19,35,25,1.4
3,2025-12-22,35,25,1.4
5,2025-12-24,35,25,1.4


In [11]:
import pandas as pd
fleet = pd.read_csv("fleet.csv")
print(fleet.columns)
print("\nOlası vinç kolonları örnek değerler:")
for c in fleet.columns:
    if any(k in c.lower() for k in ["crane","vin","vinc"]):
        print(c, fleet[c].value_counts(dropna=False).head(10))


Index(['vehicle_name', 'vehicle_id', 'vehicle_type', 'capacity_m3',
       'capacity_ton'],
      dtype='object')

Olası vinç kolonları örnek değerler:
